In [42]:
import requests
import os
import json
import time
import pandas as pd
import sys
import os
import numpy as np
from datetime import date

In [75]:
try: 
    df = pd.read_json("congressmen.json")
except Exception as e:
    print("There is an issue with the congressmen.json. Quitting.")
    sys.exit()


In [76]:
df.head()

,bioguideID,name,partyName,state,url,attribution,imageUrl,chamber,endYear,startYear
0,C000243,"Castle, Michael N.",Republican,Delaware,https://api.congress.gov/v3/member/C000243?for...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/c000243_20...,House of Representatives,2011.0,1993
1,G000590,"Green, Mark E.",Republican,Tennessee,https://api.congress.gov/v3/member/G000590?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000590_20...,House of Representatives,2025.0,2019
2,C000488,"Clay, William (Bill)",Democratic,Missouri,https://api.congress.gov/v3/member/C000488?for...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/c000488_20...,House of Representatives,2001.0,1969
3,M001233,"Messmer, Mark B.",Republican,Indiana,https://api.congress.gov/v3/member/M001233?for...,Image courtesy of the Member,https://www.congress.gov/img/member/677448630b...,House of Representatives,NaN,2025
4,M000309,"McCarthy, Carolyn",Democratic,New York,https://api.congress.gov/v3/member/M000309?for...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/m000309_20...,House of Representatives,2015.0,1997


Updating the endYear to fill NaNs with expected end years/reelection years.
Also add a column "currentMember" which is either y or n

In [ ]:
print(f"There are {len(df[df['endYear'].isna()])} NA endyears")

df['current_member'] = np.where(df['endYear'].isna(), "yes", "no")
print(f"Added yes to {len(df[df['current_member']=="yes"])} members")


#hor_mask = (df['chamber'] == "House of Representatives")
#sen_mask = (df['chamber'] == "Senate")
#hor_df = df[hor_mask]
#sen_df = df[sen_mask]

#df['num_terms'] = 0
year = date.today().year
#df.loc[hor_mask, 'num_terms'] = year + 2 - (year - df['startYear'])%2
#df.loc[sen_mask, 'num_terms'] = year + 6 - (year - df['startYear'])%6



#Replace endYear of HOR to startyear+2
df['endYear'] = np.where(
    (df['endYear'].isna()) & (df['chamber'] == 'House of Representatives'),
    year + 1 - (year - df['startYear'])%2,
    df['endYear']
)
#Replace endYear of Senate to startyear+6
df['endYear'] = np.where(
    (df['endYear'].isna()) & (df['chamber'] == 'Senate'),
    year + 5 - (year - df['startYear'])%6,
    df['endYear']
)

df['endYear'] = df['endYear'].astype(int)


print(len(df[df['endYear'].isna()]))



There are 537 NA endyears
Added yes to 537 members
0


Add the tenure

In [78]:
df['duration'] = df['endYear'] - df['startYear']

#tenure_all_time is across everyone, and across all time
df['tenure_all_time']  = df['duration'].rank(ascending=False, method='min').astype(int)
df['tenure_party_all_time'] = df.groupby('partyName')['duration'].rank(ascending=False, method='min').astype(int)

#tenure_current is just for current members, if they're not current members will be nan
df['tenure_current'] = np.where(df['current_member']=="yes", df.groupby('current_member')['duration'].rank(ascending=False,method='min'), np.nan)
df['tenure_current_party'] = np.where(df['current_member']=="yes", df.groupby(['current_member','partyName'])['duration'].rank(ascending=False,method='min'), np.nan)

df['tenure_current'] = df['tenure_current'].astype(pd.Int64Dtype())
df['tenure_current_party'] = df['tenure_current_party'].astype(pd.Int64Dtype())

df['party_all_time_count'] = df.groupby('partyName')['bioguideID'].transform('count')
df['party_current_count'] = df.groupby(['partyName','current_member'])['bioguideID'].transform('count')



In [79]:
df_test = df[df['current_member']=="yes"]

print(len(df_test))

537


In [80]:
df.sort_values(by='tenure_current', ascending=True).head()

,bioguideID,name,partyName,state,url,attribution,imageUrl,chamber,endYear,startYear,current_member,duration,tenure_all_time,tenure_party_all_time,tenure_current,tenure_current_party,party_all_time_count,party_current_count
579,G000386,"Grassley, Chuck",Republican,Iowa,https://api.congress.gov/v3/member/G000386?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/g000386_20...,Senate,2029,1981,yes,48,7,3,1,1,1423,275
321,R000395,"Rogers, Harold",Republican,Kentucky,https://api.congress.gov/v3/member/R000395?for...,Image courtesy of the Member,https://www.congress.gov/img/member/r000395_20...,House of Representatives,2027,1981,yes,46,11,4,2,2,1423,275
215,S000522,"Smith, Christopher H.",Republican,New Jersey,https://api.congress.gov/v3/member/S000522?for...,Collection of the U.S. House of Representative...,https://www.congress.gov/img/member/s000522_20...,House of Representatives,2027,1981,yes,46,11,4,2,2,1423,275
290,H000874,"Hoyer, Steny H.",Democratic,Maryland,https://api.congress.gov/v3/member/H000874?for...,Congressional Pictorial Directory,https://www.congress.gov/img/member/116_rp_md_...,House of Representatives,2027,1981,yes,46,11,8,2,1,1434,260
183,K000009,"Kaptur, Marcy",Democratic,Ohio,https://api.congress.gov/v3/member/K000009?for...,Image courtesy of the Member,https://www.congress.gov/img/member/k000009_20...,House of Representatives,2027,1983,yes,44,15,10,5,2,1434,260


In [66]:
df['name'] = df['name'].str.split(', ').str[::-1].str.join(' ')

df.head()

,bioguideID,name,partyName,state,url,attribution,imageUrl,chamber,endYear,startYear,current_member,duration,tenure_all_time,tenure_party_all_time,tenure_current,tenure_current_party
0,C000243,Michael N. Castle,Republican,Delaware,https://api.congress.gov/v3/member/C000243?for...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/c000243_20...,House of Representatives,2011,1993,no,18,559,209,<NA>,<NA>
1,G000590,Mark E. Green,Republican,Tennessee,https://api.congress.gov/v3/member/G000590?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000590_20...,House of Representatives,2025,2019,no,6,1794,845,<NA>,<NA>
2,C000488,William (Bill) Clay,Democratic,Missouri,https://api.congress.gov/v3/member/C000488?for...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/c000488_20...,House of Representatives,2001,1969,no,32,107,75,<NA>,<NA>
3,M001233,Mark B. Messmer,Republican,Indiana,https://api.congress.gov/v3/member/M001233?for...,Image courtesy of the Member,https://www.congress.gov/img/member/677448630b...,House of Representatives,2027,2025,yes,2,2485,1228,467,240
4,M000309,Carolyn McCarthy,Democratic,New York,https://api.congress.gov/v3/member/M000309?for...,Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/m000309_20...,House of Representatives,2015,1997,no,18,559,349,<NA>,<NA>
